In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Họ và tên: Nguyễn Mạnh Hưng  
MSSV: 19021295  
Lớp: INT3405E 20
Giáo viên hướng dẫn: Trần Quốc Long

### TABLE OF CONTENTS
#### 1. Mô tả bài toán và đọc dữ liệu
#### 2. Phân tích dữ liệu
#### 3. Khởi tạo các hàm xử lý và train, validate dữ liệu
#### 4. Thực thi các hàm xử lý dữ liệu
#### 5. Dự đoán đầu ra trên test.csv

### 1. Mô tả bài toán và đọc dữ liệu

#### **Mô tả bài toán**: Đây là bài toán thuộc nhóm bài toán phân lớp, với đầu vào là một câu hỏi, đầu ra là 1 nếu câu hỏi đó là spam và 1 là ngược lại

#### Đọc dữ liệu từ file train.csv và test.csv

In [ ]:
data = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
print(data.head())
test_data = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')
print(test_data.head())


### 2. Phân tích dữ liệu:
#### File csv gồm có 3 cột:  
**Cột qid:** Mã câu hỏi   
**Cột question_text:** Nội dung câu hỏi  
**Cột target:** Nhãn câu hỏi được phân loại  


In [ ]:
import torch
from torch import nn
import re
import nltk


### 3. Thiết lập các hàm:
* Xử lý dữ liệu
* Tạo dictionary
* Khởi tạo mô hình
* Chuyển đổi dữ liệu thành dataset
* Huấn luyện và đánh giá mô hình

#### **Hàm thực hiện tiền xử lý dữ liệu, bao gồm:**
* Loại bỏ số  
* Loại bỏ các dấu câu
* Loại bỏ URL  
* Loại bỏ emoji, symbol, icon  
* Chuyển tất cả các chữ viết hoa về viết thường  
* Tách từ, sử dụng thư viện nltk

In [ ]:

def _preprocess(list_sentence):
    
    # remove URL
    
    URL = r'http\S+'
    for i in range(len(list_sentence)):
        list_sentence[i] = re.sub(URL, ' ', list_sentence[i])
        
    # remove emoji, icon, symbol
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    for i in range(len(list_sentence)):
        list_sentence[i] = emoji_pattern.sub(r' ', list_sentence[i])
        
    # remove punctuation and number
    
    punc = r'[0-9\.\,\?\!\(\)\*\^\"\;\:\<\>\/\\\-\+\=]+'
    for i in range(len(list_sentence)):
        list_sentence[i] = re.sub(punc, ' ', list_sentence[i])
    
    # remove duplicate space
    list_set = []
    for sen in list_sentence:
        list_set.append(sen.split(' '))

    print('step 1 completed')
    list_set1 = []
    for i in range(len(list_set)):
        _set = []
        for j in range(len(list_set[i])):
            if list_set[i][j] != '':
                _set.append(list_set[i][j])
        list_set1.append(_set)

    list_set = list_set1.copy()
    list_sentence = []

    for _set in list_set:
        list_sentence.append(' '.join(_set))
    
    # lower sentence
    
    for i in range(len(list_sentence)):
        list_sentence[i] = list_sentence[i].lower()
    # segment word
    
    list_seq = []
    for sen in list_sentence:
        list_seq.append(nltk.word_tokenize(sen))
    
    return list_seq

#### Các hàm xây dựng bộ từ vựng dựa trên các từ tách được trong file train và test.csv, sau đó tiền hành đánh số các từ trong các câu mới được tách dựa trên vị trí các từ trong tập từ điển

In [ ]:
def count_word(list_seq):
    list_vocab = []
    for seq in list_seq:
        list_vocab.extend(seq)
    list_vocab = sorted(set(list_vocab))
    fre_word = {}
    for word in list_vocab:
        fre_word[word] = 0
    for seq in list_seq:
        for word in seq:
            fre_word[word] += 1
    return fre_word


def create_vocab(fre_word, vocab_size=70000):
    sort_word = sorted(fre_word, key=fre_word.get)
    dict_word = {}
    dict_word['[unk]'] = 0
    index = 1
    for i in range(len(sort_word) - vocab_size + 1, len(sort_word)):
        dict_word[sort_word[i]] = index
        index += 1
    return dict_word


def index(word, dict_word):
    if word in dict_word.keys():
        return dict_word[word]
    return dict_word['[unk]']

def word_to_index(list_seq, dict_word):
    list_index = []
    for seq in list_seq:
        set_index = []
        for word in seq:
            set_index.append(index(word, dict_word))
        list_index.append(set_index)
    return list_index

#### Hàm thực hiện loại bỏ các câu bị rỗng sau khi tiền xử lý

In [ ]:
def remove_empty(list_seq, list_label):
    new_seqs = []
    new_labels = []
    for i in range(len(list_seq)):
        if len(list_seq[i]) > 0:
            new_seqs.append(list_seq[i])
            new_labels.append(list_label[i])

    return [new_seqs, new_labels]

#### Tiền hành chuyển dữ liệu đọc được thành các list để tiện xử lý

In [ ]:



list_sentence = []
for i in range(len(data)):
    list_sentence.append(str(data['question_text'].loc[i]))


list_label = []
for i in range(len(data)):
    list_label.append(float(data['target'].loc[i]))
    
test_sentence = []
for i in range(len(test_data)):
    test_sentence.append(str(test_data['question_text'].loc[i]))

#### Tiền hành định lượng phân bố số lượng các câu theo các nhãn tương ứng trong tập train.csv

In [ ]:
import matplotlib.pyplot as plt
count0 = 0
count1 = 0
for label in list_label:
    if int(label) == 0:
        count0 += 1
    else: count1 += 1
print('So luong nhan 0 la ', count0)
print('So luong nhan 1 la ', count1)

left = [1, 2]

# heights of bars
height = [count0, count1]
 
# labels for bars
tick_label = [0, 1]
 
# plotting a bar chart
plt.bar(left, height, tick_label = tick_label,
        width = 0.8, color = ['red', 'green'])
 
# naming the x-axis
plt.xlabel('label')
# naming the y-axis
plt.ylabel('the quantities')
# plot title
plt.title('Number each label in the orgin dataset')
 
# function to show the plot
plt.show()

Nhìn vào đồ thị có thể thấy được label 1 chỉ chiếm 6 % tổng số nhãn trong cả tập train data, điều này khiến mô hình bị imbalanced nặng, cái mà khiến mô hình dễ bias với các nhãn còn lại. Do đó, cần phải tìm cách khắc phục vấn đề này.  
Có hai loại phương pháp cơ bản nhằm giải quyết vấn đề này, đó là oversampling và undersampling.Thực chất hai phương pháp vừa kể chỉ đơn giản là bồi thêm dữ liệu ở nhãn ít hơn hay cắt bớt giữ lieuj ở nhãn cao hơn sao cho lương dữ liệu ở các nhãn là như nhau. Ở đây, tôi sẽ tiến hành phương pháp undersampling, được thực hiện qua hàm _create_dataset.  
Tại hàm _create_dataset, tôi sẽ cắt bớt dữ liệu ở nhãn 0 một cách ngẫu nhiên, giữ lại một lượng dữ liệu ngang với lương dữ liệu ở label 1. Sau đó gộp hai dữ liệu label 0 mới và dữ liệu label 1 thành tập data để tiến hành training và testing. 

#### Khởi tạo mô hình mạng nơ-ron theo cách tiếp cận của lớp encoder của transformer, với 8 đầu multi-head attentions.

In [ ]:

import math
#initialize transformer class
class self_transformer(nn.Module):
        def __init__(self, input_dim, output_dim):
            super(self_transformer, self).__init__()
            self.linear1 = nn.Linear(input_dim, output_dim)
            self.linear2 = nn.Linear(input_dim, output_dim)
            self.linear3 = nn.Linear(input_dim, output_dim)
            self.softmax = nn.Softmax(-1)
        def forward(self, _input, output_dim):
            question = self.linear1(_input) # 30x64
            key = self.linear2(_input) # 30x64
            value = self.linear3(_input) # 30x64
            self_attention_ = torch.matmul(key, torch.transpose(question, - 2, -1))/math.sqrt(output_dim)# 30x30
            self_attention = self.softmax(self_attention_) # 30x30
            # print(self_attention.shape)
            self_output = torch.matmul(self_attention, value)
            # print(self_output.shape)
            return self_output

class multi_head_transformer(nn.Module):
    def __init__(self, input_dim, output_dim, num_class):
        super(multi_head_transformer, self).__init__()
        self.linear = []
        for i in range(num_class):
            self.linear.append(nn.Linear(input_dim, output_dim))
    def forward(self, multi,output_dim):
        result = 0.0
        for i in range(len(multi)):
            result += self.linear[i](multi[i])
        result = torch.flatten(result, start_dim=-2)
        result = result.mean(-1)
        result = torch.sigmoid(result)
        return result


def transform(_input, model_1, model_2, output_dim_encode,output_dim_decode):
    encoders = []
    for i in range(0, 8):
        encoders.append(model_1[i](_input, output_dim_encode))
    decoder = model_2(encoders, output_dim_decode)
    return decoder



#### Tạo các hàm để tiến hành chuyển các feature, label thành các dataset để tiến hành đưa vào mô hình, trong đó:
* Hàm padding tiến hành đưa các features về cùng kích cỡ chiều dài  
* Hàm create_pos tạo ma trận position encoding, giúp biểu thị ý nghĩa về vị trí các từ trong câu của các features  
* Hàm process_data_to_input sẽ nhúng các feature, sử dụng hàm nhúng word2vec kết hợp với position embedding vừa kể trên  
* Hàm create_transformer_model sẽ khởi tạo các tham số cần thiết của mô hình traing, gôm tạo mạng neuron, các hàm mất mát cũng như hàm tính toán xuống đồi  
* Hàm split_dataset sẽ tiến hành chia dataset thành tập train và tập validation một cách ngẫu nhiên theo một tỉ lệ nhất định  
* Hàm train_and_validation sẽ tiến hành training và sau đó sẽ tính toán accuarcy trên tập validation

In [ ]:

from keras.preprocessing.sequence import pad_sequences
from torch import nn
import random
import math
from torch.utils.data import *


#initialize some parameter for padding

def padding(list_seq, max_len):
    return torch.from_numpy(pad_sequences(sequences=list_seq, maxlen=max_len))


#processing pads into input before putting in transformer models


def create_pos(length, embed_dim=24):
    position_encode = torch.zeros((length, embed_dim))
    for i in range(length):
        for j in range(embed_dim):
            if j % 2 == 1:
                position_encode[i][j] = math.sin(float(i+1)/10000**(float(j+1)/embed_dim))
            else:
                position_encode[i][j] = math.cos(float(i+1)/10000**(float(j)/embed_dim))
    return position_encode


def process_data_to_input(data, embed_model, max_len):
    return embed_model(data) + create_pos(max_len)
 
 
def create_transform_model(vocab_size, embed_dim):    
    model_1 = []
    for i in range(8):
        model_1.append(self_transformer(input_dim=embed_dim, output_dim = 24))
    model_2 = multi_head_transformer(input_dim = 24, output_dim=embed_dim, num_class=8)
    embed_model = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim)
    
    criterion = nn.L1Loss()
    



    optimizer = torch.optim.Adam([{'params':model_1[0].parameters()}, 
                                {'params':model_1[1].parameters()}, 
                                {'params':model_1[2].parameters()}, 
                                {'params':model_1[3].parameters()}, 
                                {'params':model_1[4].parameters()}, 
                                {'params':model_1[5].parameters()}, 
                                {'params':model_1[6].parameters()}, 
                                {'params':model_1[7].parameters()}, 
                                {'params':embed_model.parameters()},
                                {'params':model_2.parameters()}], lr=0.00108)
    
    return [model_1, model_2, embed_model, criterion, optimizer]




def create_dataset(_input, label):
    return [TensorDataset(_input, label), label.shape[0]]
    # processed data

def _create_dataset(feature, label):
    po_feature = []
    ne_feature = []
    po_label = []
    ne_label = []
    for i in range(feature.shape[0]):
        if int(label[i]) == 0:
            ne_feature.append(feature[i].unsqueeze(0))
            ne_label.append(label[i].unsqueeze(0))
        else:
            po_feature.append(feature[i].unsqueeze(0))
            po_label.append(label[i].unsqueeze(0))
    min_count = min(len(ne_label), len(po_label))
    print(min_count)

    random_po = random.sample(range(0, len(po_label)), min_count)
    random_ne = random.sample(range(0, len(ne_label)), min_count)

    new_feature = []
    new_label = []

    for i in range(2*min_count):
        if i % 2 == 0:
            new_feature.append(po_feature[random_po[int(i/2)]])
            new_label.append(po_label[random_po[int(i/2)]])
        else:
            new_feature.append(ne_feature[random_ne[int((i-1)/2)]])
            new_label.append(ne_label[random_ne[int((i-1)/2)]])
    new_feature = torch.cat(new_feature)
    new_label = torch.cat(new_label)
    print(new_feature.shape)
    print(new_label.shape)

    return [TensorDataset(new_feature, new_label), 2*min_count]


def split_dataset(data, data_size, percentage_attr):
    train_size = int(data_size * percentage_attr)
    test_size = data_size - train_size
    input_train, input_test = random_split(data, [train_size, test_size])
    train_loader = DataLoader(input_train.dataset, batch_size=16, shuffle=True)
    test_loader = DataLoader(input_test.dataset, batch_size=16, shuffle=False)
    return [train_loader, test_loader]

def train_and_validation(train_loader, test_loader, model_1, model_2, optimizer, criterion, encode_dim=24, decode_dim=16, num_epochs = 10):
    validation_acc = []
    for epoch in range(num_epochs):
        for i, mini_data in enumerate(train_loader):
            inputs, labels = mini_data
            embed_layer = process_data_to_input(inputs, embed_model, padding_size)
            optimizer.zero_grad()
            outputs = transform(embed_layer, model_1, model_2, output_dim_encode=encode_dim, output_dim_decode=decode_dim)
            
            loss = criterion(outputs, labels)
            loss.backward(retain_graph=True)
            optimizer.step()
            # print('hello')

            if i % 300 == 0:
                print(loss.data)
            
        print('complete epoch ' + str(epoch) +' completed')
            
        count = 0
        total_count = 0    
        for i, mini_data in enumerate(test_loader):
            
            inputs, labels = mini_data
            embed_layer = process_data_to_input(inputs, embed_model, padding_size)
            outputs = transform(embed_layer, model_1, model_2, output_dim_encode=encode_dim, output_dim_decode=decode_dim)
            for j in range(outputs.shape[0]):
                total_count += 1
                if abs(outputs[j].data - labels[j].data) < 0.5:
                    count += 1
        
        validation_acc.append(count/total_count)
    predict_labels = []
    real_labels = []
    for i, mini_data in enumerate(test_loader):
        
        inputs, labels = mini_data
        embed_layer = process_data_to_input(inputs, embed_model, padding_size)
        outputs = transform(embed_layer, model_1, model_2, output_dim_encode=encode_dim, output_dim_decode=decode_dim)
        for j in range(outputs.shape[0]):
            predict_labels.append(outputs[j].data)
            real_labels.append(labels[j].data)
    for i in range(len(predict_labels)):
        if predict_labels[i] > 0.5:
            predict_labels[i] = 1
        else:
            predict_labels[i] = 0
    return [validation_acc, predict_labels, real_labels]





        
        

### 4. Thực thi các hàm xử lý dữ liệu và chạy mô hình

Sau khi khởi tạo các hàm thích hợp, bây giờ sẽ áp dụng các hàm đấy để xử lý dữ liệu và huấn luyện mô hình cũng như kiểm thử

In [ ]:
nltk.download('punkt')

In [ ]:
list_seq = _preprocess(list_sentence)
test_seq = _preprocess(test_sentence)

Dưới đây là ví dụ một vài câu đã được xử lý

In [ ]:
random_taken = random.sample(range(0, len(list_seq)), 5)
orgin_sentence = []
pos_preprocessed_sentence = []
for i in range(len(random_taken)):
    orgin_sentence.append(list_sentence[random_taken[i]])
    pos_preprocessed_sentence.append(list_seq[random_taken[i]])
example = pd.DataFrame({'Trước khi xử lý': orgin_sentence})
example['Sau khi xử lý'] = pos_preprocessed_sentence
example

    

In [ ]:
all_seq = list_seq.copy()
all_seq.extend(test_seq)

In [ ]:
list_seq, list_label = remove_empty(list_seq, list_label)
print('Số câu còn lại sau khi loại bỏ các câu rỗng ', len(list_seq))
print('Số nhãn còn lại sau khi loại bỏ các nhãn ứng với câu rỗng ', len(list_label))


In [ ]:

fre_word = count_word(all_seq)
dict_word = create_vocab(fre_word, vocab_size=70000)
list_index = word_to_index(list_seq, dict_word)


#### Khởi tạo các tham số cần thiết, với:  
* vocab_size: Kích thước từ vựng
* batch_size: Kích thước mini batch  
* embed_size: Kích thước nhúng từ 
* padding_size: Kích thước padding, các câu sẽ được căn chỉnh về độ dài cố định đúng bằng nó  
* percentage_attr: Tỉ lệ lượng dữ liệu làm training và validating lấy ra từ tập data gốc


In [ ]:
vocab_size = 70000
batch_size = 16
embed_size = 24
padding_size = 16
percentage_attr = 0.96

In [ ]:
list_label = torch.tensor(list_label)

In [ ]:
model_1, model_2, embed_model, criterion, optimizer = create_transform_model(vocab_size, embed_size)







    



In [ ]:
padding_seq = padding(list_index, padding_size)

In [ ]:
processed_data, data_size = _create_dataset(padding_seq, list_label)



Như đã đề cập hàm _create_dataset sẽ tiến hành undersampling dữ liệu sau đó chuyển thành dataset, ta cùng thống kê số lượng cặp feature và label ứng với nhãn 0 và 1 còn lại

In [ ]:



left = [1, 2]

# heights of bars
height = [data_size/2, data_size/2]
 
# labels for bars
tick_label = [0, 1]
 
# plotting a bar chart
plt.bar(left, height, tick_label = tick_label,
        width = 0.8, color = ['red', 'green'])
 
# naming the x-axis
plt.xlabel('label')
# naming the y-axis
plt.ylabel('the quantities')
# plot title
plt.title('Number each label in the updated dataset')
 
# function to show the plot
plt.show()

In [ ]:
train_loader, test_loader = split_dataset(processed_data, data_size,percentage_attr)


#### In ra accuracy, p score, r score, f1 score cho các nhãn

In [ ]:
acc, predict_labels, real_labels = train_and_validation(train_loader, test_loader, model_1, model_2, optimizer, criterion)
print('Accuracy của mô hình trên tập validation là ', acc[len(acc) - 1], '\n')
tp = [0.0, 0.0]
po = [0.0, 0.0]
fn = [0.0, 0.0]
p_score = [0.0, 0.0]
r_score = [0.0, 0.0]
f1_score = [0.0, 0.0]
for label in range(2):
    for i in range(len(predict_labels)):
        if int(predict_labels[i]) == label:
            po[label] += 1
            if int(predict_labels[i]) == int(real_labels[i]):
                tp[label] += 1
        else:
            if int(predict_labels[i]) != int(real_labels[i]):
                fn[label] += 1
    p_score[label] = tp[label] / (po[label] + 1e-4)
    r_score[label] = tp[label] / (tp[label] + fn[label] + 1e-4)
    f1_score[label] = 2 * p_score[label] * r_score[label] / (p_score[label] + r_score[label])
print('Với nhãn 0:\nĐiểm precision là ', p_score[0], '\nĐiểm recall là ', r_score[0], '\nĐiểm f1 la ', f1_score[0], '\n')
print('Với nhãn 1:\nĐiểm precision là ', p_score[1], '\nĐiểm recall là ', r_score[1], '\nĐiểm f1 la ', f1_score[1], '\n')

### 5. Dự đoán đầu ra trên test.csv

#### Now turn to predicting label for test.csv, the result is print in submission.csv

In [ ]:
test_index = word_to_index(test_seq, dict_word)

In [ ]:
padding_test = padding(test_index, padding_size)

In [ ]:
padding_test = process_data_to_input(padding_test, embed_model, padding_size)

In [ ]:
def predict(features, model_1, model_2, optimizer, criterion, encode_dim=24, decode_dim=16):
    list_label = []
    for i in range(features.shape[0]):
        list_label.append( transform(features[i], model_1, model_2, output_dim_encode=encode_dim, output_dim_decode=decode_dim).data)
    result = []
    for i in range(len(list_label)):
        if list_label[i] < 0.5:
            result.append(0)
        else: result.append(1)
    return result

In [ ]:
result = predict(padding_test, model_1, model_2, optimizer, criterion, encode_dim=24, decode_dim=16)

In [ ]:
print(result[0:100])

In [ ]:
submit_data = pd.DataFrame({'qid': test_data['qid'].values})

In [ ]:
submit_data['prediction'] = result
submit_data.to_csv('submission.csv', index=False)
submit_data